In [30]:
import numpy as np
import pandas as pd

import pickle
import nltk
import nltk
from nltk.corpus import stopwords
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))
stopwords = set(stopwords.words('english'))

import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from sklearn.pipeline import make_pipeline
from nltk import word_tokenize
from statistics import mode
from tensorflow.keras.preprocessing.text import one_hot
from nltk.classify import ClassifierI
from statistics import mode

import tensorflow as tf  

import pickle

In [32]:
model_filenames = [
    'nltk_2_naive_bayes_model_40.pkl',
    'nltk_naive_bayes_model_42.pkl',
    'sklearn_BNB_57.pkl',
    'sklearn_Lsvc_model_60.pkl',
    'sklearn_m_LRC_61.pkl',
    'sklearn_m_SGD_61.pkl',
    'sklearn_naive_bayes_model_52.pkl',
    'sklearn_svc_model_61.pkl'
]

In [34]:
classifiers = []
for model_filename in model_filenames:
    with open(model_filename, 'rb') as f:
        classifier = pickle.load(f)
        classifiers.append(classifier)


In [35]:
# Load Keras model
h5_model = load_model('trained_model_68_.h5')
classifiers.append(h5_model)


In [37]:
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)


In [13]:
# Preprocess function for NLTK models
def preprocess_nltk(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token not in stopwords_set]
    return {word: True for word in tokens}


In [40]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, sentence):
        votes = []

        # Transform the input sentence into TF-IDF features
        features_tfidf = vectorizer.transform([sentence])  # Ensure this is a 2D array
        processed_features = self.preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                # Predict using the scikit-learn classifiers
                prediction = clf.predict(features_tfidf)  # No need to index; already returns an array
                votes.append(str(prediction[0]))  # Append the prediction
            elif hasattr(clf, 'classify'):
                # Predict using the NLTK classifiers
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))
            else:
                # Handle the Keras model
                cleaned_sentence = self.clean_new_text(sentence)
                # Make sure to format input as needed for Keras
                prediction = clf.predict(np.array([cleaned_sentence]))  # Adjust this as necessary
                predicted_class = np.argmax(prediction, axis=1)[0]
                votes.append(str(predicted_class))  # Append the predicted class

        # Return the most common vote
        return mode(votes)

    def preprocess_nltk(self, text):
        tokens = nltk.word_tokenize(text)
        tokens = [token.lower() for token in tokens if token.isalpha() and token not in stopwords_set]
        return {word: True for word in tokens}

    def confidence(self, sentence):
        votes = []
        features_tfidf = vectorizer.transform([sentence])
        processed_features = self.preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                prediction = clf.predict(features_tfidf)
                votes.append(str(prediction[0]))
            elif hasattr(clf, 'classify'):
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

    def clean_new_text(self, text):
        stemmer = nltk.stem.PorterStemmer()
        text = re.sub("[^a-zA-Z]", " ", text)
        text = text.lower()
        words = text.split()
        words = [stemmer.stem(word) for word in words if word not in stopwords_set]
        cleaned_text = " ".join(words)
        return cleaned_text

In [42]:
# Example usage
voting_classifier = VoteClassifier(*classifiers)

In [44]:
emotion_labels = ["Anger", "Disgust", "Fear", "Happiness", "Sadness", "Surprise", "Neutral"]

sentence = "I am feeling really Happy today!"
predicted_emotion = voting_classifier.classify(sentence)
confidence = voting_classifier.confidence(sentence)

# Convert the predicted emotion index from string to integer
predicted_index = int(predicted_emotion)

# Access the corresponding emotion label using the index
if 0 <= predicted_index < len(emotion_labels):
    corresponding_emotion = emotion_labels[predicted_index]
    print(f"Predicted emotion (index {predicted_index}): {corresponding_emotion}")
else:
    print(f"Index {predicted_index} is out of range for emotion_labels.")

print(f"Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 983ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 763ms/step
Predicted emotion (index 3): Happiness
Confidence: 0.78


In [46]:
with open('voting_classifier.pkl', 'wb') as f:
    pickle.dump(voting_classifier, f)

In [21]:
# Load all models
model_filenames = [
    'nltk_2_naive_bayes_model_40.pkl',
    'nltk_naive_bayes_model_42.pkl',
    'sklearn_BNB_57.pkl',
    'sklearn_Lsvc_model_60.pkl',
    'sklearn_m_LRC_61.pkl',
    'sklearn_m_SGD_61.pkl',
    'sklearn_naive_bayes_model_52.pkl',
    'sklearn_svc_model_61.pkl'
]

classifiers = []
for model_filename in model_filenames:
    with open(model_filename, 'rb') as f:
        classifier = pickle.load(f)
    classifiers.append(classifier)

# Load the Keras model
h5_model = load_model('trained_model_68_.h5')
classifiers.append(h5_model)

# Load the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# Define the VoteClassifier class (as provided)

# Create the voting classifier
voting_classifier = VoteClassifier(*classifiers)

# Save the ensemble model to a .pkl file
with open('voting_classifier.pkl', 'wb') as f:
    pickle.dump(voting_classifier, f)

In [28]:
# Load trained models
model_filenames = [
    'nltk_2_naive_bayes_model_40.pkl',
    'nltk_naive_bayes_model_42.pkl',
    'sklearn_BNB_57.pkl',
    'sklearn_Lsvc_model_60.pkl',
    'sklearn_m_LRC_61.pkl',
    'sklearn_m_SGD_61.pkl',
    'sklearn_naive_bayes_model_52.pkl',
    'sklearn_svc_model_61.pkl'
]

classifiers = []
for model_filename in model_filenames:
    with open(model_filename, 'rb') as f:
        classifier = pickle.load(f)
    classifiers.append(classifier)

# Load Keras model
h5_model = load_model('trained_model_68_.h5')
classifiers.append(h5_model)

# Load TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# Define a wrapper class for the voting classifier
class VotingEnsemble:
    def __init__(self, classifiers, vectorizer):
        self.classifiers = classifiers
        self.vectorizer = vectorizer

    def preprocess_text(self, text):
        # Tokenize and clean the input text
        text = re.sub("[^a-zA-Z]", " ", text).lower()
        words = nltk.word_tokenize(text)
        words = [word for word in words if word not in stopwords_set]
        return " ".join(words)

    def predict(self, sentence):
        # Preprocess the input
        preprocessed_sentence = self.preprocess_text(sentence)
        
        # Transform using the vectorizer
        features_tfidf = self.vectorizer.transform([preprocessed_sentence])
        
        votes = []
        for clf in self.classifiers:
            if hasattr(clf, 'predict'):
                prediction = clf.predict(features_tfidf)
                votes.append(str(prediction[0]))
            elif hasattr(clf, 'classify'):
                # Adjust for NLTK classifiers
                tokens = {word: True for word in preprocessed_sentence.split()}
                prediction = clf.classify(tokens)
                votes.append(str(prediction))
            else:
                # Handle Keras model
                prediction_input = self.vectorizer.transform([preprocessed_sentence]).toarray()
                prediction = clf.predict(prediction_input)
                predicted_class = np.argmax(prediction, axis=1)[0]
                votes.append(str(predicted_class))
        
        # Return the most common prediction
        return mode(votes)

# Create the ensemble object
voting_ensemble = VotingEnsemble(classifiers, vectorizer)

# Save the entire ensemble including preprocessing to a .pkl file
with open('ok.pkl', 'wb') as f:
    pickle.dump(voting_ensemble, f)

In [26]:
with open('voting_ensemble_with_preprocessing.pkl', 'rb') as file:
    text_model = pickle.load(file)

C:\Users\ssttg\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# _______ NOT USEFULL ______________________

In [38]:
import pickle
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from statistics import mode
from nltk.classify import ClassifierI
from sklearn.naive_bayes import BernoulliNB
from keras.models import load_model
import re


from nltk.corpus import stopwords

# Set stopwords
stopwords_set = set(stopwords.words('english'))

# Load your trained models
model_filenames = [
    'nltk_2_naive_bayes_model_40.pkl',
    'nltk_naive_bayes_model_42.pkl',
    'sklearn_BNB_57.pkl',
    'sklearn_Lsvc_model_60.pkl',
    'sklearn_m_LRC_61.pkl',
    'sklearn_m_SGD_61.pkl',
    'sklearn_naive_bayes_model_52.pkl',
    'sklearn_svc_model_61.pkl'
]

classifiers = []
for model_filename in model_filenames:
    with open(model_filename, 'rb') as f:
        classifier = pickle.load(f)
        classifiers.append(classifier)

# Load Keras model
h5_model = load_model('trained_model_68_.h5')
classifiers.append(h5_model)

with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
    
# Preprocess function for NLTK models
def preprocess_nltk(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token not in stopwords_set]
    return {word: True for word in tokens}

# Voting classifier
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, sentence):
        votes = []

        # Transform the input sentence into TF-IDF features
        features_tfidf = vectorizer.transform([sentence])  # Ensure this is a 2D array
        processed_features = preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                # Predict using the scikit-learn classifiers
                prediction = clf.predict(features_tfidf)  # No need to index; already returns an array
                votes.append(str(prediction[0]))  # Append the prediction
            elif hasattr(clf, 'classify'):
                # Predict using the NLTK classifiers
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))
            else:
                # Handle the Keras model
                cleaned_sentence = self.clean_new_text(sentence)
                # Make sure to format input as needed for Keras
                prediction = clf.predict(np.array([cleaned_sentence]))  # Adjust this as necessary
                predicted_class = np.argmax(prediction, axis=1)[0]
                votes.append(str(predicted_class))  # Append the predicted class

        # Return the most common vote
        return mode(votes)

    def confidence(self, sentence):
        votes = []
        features_tfidf = vectorizer.transform([sentence])
        processed_features = preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                prediction = clf.predict(features_tfidf)
                votes.append(str(prediction[0]))
            elif hasattr(clf, 'classify'):
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

    def clean_new_text(self, text):
        stemmer = nltk.stem.PorterStemmer()
        text = re.sub("[^a-zA-Z]", " ", text)
        text = text.lower()
        words = text.split()
        words = [stemmer.stem(word) for word in words if word not in stopwords_set]
        cleaned_text = " ".join(words)
        return cleaned_text

# Example usage
voting_classifier = VoteClassifier(*classifiers)

emotion_labels = ["Anger", "Disgust", "Fear", "Happiness", "Sadness", "Surprise", "Neutral"]

sentence = "I am feeling really Happy today!"
predicted_emotion = voting_classifier.classify(sentence)
confidence = voting_classifier.confidence(sentence)

# Convert the predicted emotion index from string to integer
predicted_index = int(predicted_emotion)

# Access the corresponding emotion label using the index
if 0 <= predicted_index < len(emotion_labels):
    corresponding_emotion = emotion_labels[predicted_index]
    print(f"Predicted emotion (index {predicted_index}): {corresponding_emotion}")
else:
    print(f"Index {predicted_index} is out of range for emotion_labels.")

print(f"Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 954ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 644ms/step
Predicted emotion (index 3): Happiness
Confidence: 0.78


# __________________________________________________________________

In [13]:
import pickle
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from statistics import mode
from nltk.classify import ClassifierI
from sklearn.naive_bayes import BernoulliNB
from keras.models import load_model
import re


from nltk.corpus import stopwords

# Set stopwords
stopwords_set = set(stopwords.words('english'))

# Load your trained models
model_filenames = [
    'nltk_2_naive_bayes_model_40.pkl',
    'nltk_naive_bayes_model_42.pkl',
    'sklearn_BNB_57.pkl',
    'sklearn_Lsvc_model_60.pkl',
    'sklearn_m_LRC_61.pkl',
    'sklearn_m_SGD_61.pkl',
    'sklearn_naive_bayes_model_52.pkl',
    'sklearn_svc_model_61.pkl'
]

classifiers = []
for model_filename in model_filenames:
    with open(model_filename, 'rb') as f:
        classifier = pickle.load(f)
        classifiers.append(classifier)

# Load Keras model
h5_model = load_model('trained_model_68_.h5')
classifiers.append(h5_model)

with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
    
# Preprocess function for NLTK models
def preprocess_nltk(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token not in stopwords_set]
    return {word: True for word in tokens}

# Voting classifier
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, sentence):
        votes = []

        # Transform the input sentence into TF-IDF features
        features_tfidf = vectorizer.transform([sentence])  # Ensure this is a 2D array
        processed_features = preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                # Predict using the scikit-learn classifiers
                prediction = clf.predict(features_tfidf)  # No need to index; already returns an array
                votes.append(str(prediction[0]))  # Append the prediction
            elif hasattr(clf, 'classify'):
                # Predict using the NLTK classifiers
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))
            else:
                # Handle the Keras model
                cleaned_sentence = self.clean_new_text(sentence)
                # Make sure to format input as needed for Keras
                prediction = clf.predict(np.array([cleaned_sentence]))  # Adjust this as necessary
                predicted_class = np.argmax(prediction, axis=1)[0]
                votes.append(str(predicted_class))  # Append the predicted class

        # Return the most common vote
        return mode(votes)

    def confidence(self, sentence):
        votes = []
        features_tfidf = vectorizer.transform([sentence])
        processed_features = preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                prediction = clf.predict(features_tfidf)
                votes.append(str(prediction[0]))
            elif hasattr(clf, 'classify'):
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

    def clean_new_text(self, text):
        stemmer = nltk.stem.PorterStemmer()
        text = re.sub("[^a-zA-Z]", " ", text)
        text = text.lower()
        words = text.split()
        words = [stemmer.stem(word) for word in words if word not in stopwords_set]
        cleaned_text = " ".join(words)
        return cleaned_text

# Example usage
voting_classifier = VoteClassifier(*classifiers)

sentence = "I am feeling really happy today!"
predicted_emotion = voting_classifier.classify(sentence)
confidence = voting_classifier.confidence(sentence)

print(f"Predicted emotion: {predicted_emotion}")
print(f"Confidence: {confidence:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 934ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 657ms/step
Predicted emotion: 3
Confidence: 0.78


In [40]:
# emotion_predictor.py

import pickle
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from statistics import mode
from nltk.classify import ClassifierI
from sklearn.naive_bayes import BernoulliNB
from keras.models import load_model
import re
from nltk.corpus import stopwords

# Set stopwords
stopwords_set = set(stopwords.words('english'))

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, sentence):
        votes = []
        features_tfidf = vectorizer.transform([sentence])  # Ensure this is a 2D array
        processed_features = self.preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                prediction = clf.predict(features_tfidf)
                votes.append(str(prediction[0]))
            elif hasattr(clf, 'classify'):
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))
            else:
                cleaned_sentence = self.clean_new_text(sentence)
                prediction = clf.predict(np.array([cleaned_sentence]))
                predicted_class = np.argmax(prediction, axis=1)[0]
                votes.append(str(predicted_class))

        return mode(votes)

    def confidence(self, sentence):
        votes = []
        features_tfidf = vectorizer.transform([sentence])
        processed_features = self.preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                prediction = clf.predict(features_tfidf)
                votes.append(str(prediction[0]))
            elif hasattr(clf, 'classify'):
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

    def clean_new_text(self, text):
        stemmer = nltk.stem.PorterStemmer()
        text = re.sub("[^a-zA-Z]", " ", text)
        text = text.lower()
        words = text.split()
        words = [stemmer.stem(word) for word in words if word not in stopwords_set]
        cleaned_text = " ".join(words)
        return cleaned_text

    def preprocess_nltk(self, text):
        tokens = nltk.word_tokenize(text)
        tokens = [token.lower() for token in tokens if token.isalpha() and token not in stopwords_set]
        return {word: True for word in tokens}

# Load models and vectorizer
def load_models(model_filenames, vectorizer_filename):
    classifiers = []
    for model_filename in model_filenames:
        with open(model_filename, 'rb') as f:
            classifier = pickle.load(f)
            classifiers.append(classifier)

    # Load Keras model
    h5_model = load_model('trained_model_68_.h5')
    classifiers.append(h5_model)

    # Load TF-IDF vectorizer
    with open(vectorizer_filename, 'rb') as f:
        vectorizer = pickle.load(f)
    
    return classifiers, vectorizer

# Function to predict emotion
def predict_emotion(sentence):
    emotion_labels = ["Anger", "Disgust", "Fear", "Happiness", "Sadness", "Surprise", "Neutral"]
    
    # Load models and vectorizer
    model_filenames = [
        'nltk_2_naive_bayes_model_40.pkl',
        'nltk_naive_bayes_model_42.pkl',
        'sklearn_BNB_57.pkl',
        'sklearn_Lsvc_model_60.pkl',
        'sklearn_m_LRC_61.pkl',
        'sklearn_m_SGD_61.pkl',
        'sklearn_naive_bayes_model_52.pkl',
        'sklearn_svc_model_61.pkl'
    ]
    
    classifiers, vectorizer = load_models(model_filenames, 'tfidf_vectorizer.pkl')
    
    # Initialize the voting classifier
    voting_classifier = VoteClassifier(*classifiers)

    # Make prediction
    predicted_emotion = voting_classifier.classify(sentence)
    confidence = voting_classifier.confidence(sentence)

    # Convert the predicted emotion index from string to integer
    predicted_index = int(predicted_emotion)

    # Access the corresponding emotion label using the index
    if 0 <= predicted_index < len(emotion_labels):
        corresponding_emotion = emotion_labels[predicted_index]
        print(f"Predicted emotion (index {predicted_index}): {corresponding_emotion}")
    else:
        print(f"Index {predicted_index} is out of range for emotion_labels.")

    print(f"Confidence: {confidence:.2f}")

# Example usage
if __name__ == "__main__":
    sentence = "I am feeling really happy today!"
    predict_emotion(sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 903ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step
Predicted emotion (index 3): Happiness
Confidence: 0.78


In [52]:
import pickle
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from statistics import mode
from nltk.classify import ClassifierI
from keras.models import load_model
import re
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import pandas as pd

# Set stopwords
stopwords_set = set(stopwords.words('english'))

# Load your trained models
model_filenames = [
    'nltk_2_naive_bayes_model_40.pkl',
    'nltk_naive_bayes_model_42.pkl',
    'sklearn_BNB_57.pkl',
    'sklearn_Lsvc_model_60.pkl',
    'sklearn_m_LRC_61.pkl',
    'sklearn_m_SGD_61.pkl',
    'sklearn_naive_bayes_model_52.pkl',
    'sklearn_svc_model_61.pkl'
]

classifiers = []
for model_filename in model_filenames:
    with open(model_filename, 'rb') as f:
        classifier = pickle.load(f)
        classifiers.append(classifier)

# Load Keras model
h5_model = load_model('trained_model_68_.h5')
classifiers.append(h5_model)

# Load TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
    tokenizer = vectorizer


# Maximum length used during training (adjust this as needed)
max_length = 100

# Preprocess function for NLTK models
def preprocess_nltk(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token not in stopwords_set]
    return {word: True for word in tokens}

# Voting classifier
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, sentence):
        votes = []

        # Transform the input sentence into TF-IDF features
        features_tfidf = vectorizer.transform([sentence])
        processed_features = preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                prediction = clf.predict(features_tfidf)
                votes.append(str(prediction[0]))
            elif hasattr(clf, 'classify'):
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))
            else:
                # Handle the Keras model
                cleaned_sentence = self.clean_new_text(sentence)
                prediction_input = self.prepare_keras_input(cleaned_sentence)
                prediction = clf.predict(prediction_input)
                predicted_class = np.argmax(prediction, axis=1)[0]
                votes.append(str(predicted_class))

        # Return the most common vote
        return mode(votes)

    def confidence(self, sentence):
        votes = []
        features_tfidf = vectorizer.transform([sentence])
        processed_features = preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                prediction = clf.predict(features_tfidf)
                votes.append(str(prediction[0]))
            elif hasattr(clf, 'classify'):
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

    def clean_new_text(self, text):
        stemmer = nltk.stem.PorterStemmer()
        text = re.sub("[^a-zA-Z]", " ", text)
        text = text.lower()
        words = text.split()
        words = [stemmer.stem(word) for word in words if word not in stopwords_set]
        cleaned_text = " ".join(words)
        return cleaned_text

    def prepare_keras_input(self, text):
        sequences = tokenizer.texts_to_sequences([text])
        padded_sequences = pad_sequences(sequences, maxlen=max_length)
        return padded_sequences

# Example usage
voting_classifier = VoteClassifier(*classifiers)

emotion_labels = ["Anger", "Disgust", "Fear", "Happiness", "Sadness", "Surprise", "Neutral"]

def predict_emotion(sentence):
    # Make prediction
    predicted_emotion = voting_classifier.classify(sentence)
    confidence = voting_classifier.confidence(sentence)

    # Convert the predicted emotion index from string to integer
    predicted_index = int(predicted_emotion)

    # Access the corresponding emotion label using the index
    if 0 <= predicted_index < len(emotion_labels):
        corresponding_emotion = emotion_labels[predicted_index]
        print(f"Predicted emotion (index {predicted_index}): {corresponding_emotion}")
    else:
        print(f"Index {predicted_index} is out of range for emotion_labels.")

    print(f"Confidence: {confidence:.2f}")

    return corresponding_emotion, confidence

def evaluate_accuracy(val_data_path):
    val_data = pd.read_csv(val_data_path)
    total = len(val_data)
    correct = 0

    for _, row in val_data.iterrows():
        sentence = row['Sentence']
        true_emotion = row['Emotion']
        predicted_emotion, _ = predict_emotion(sentence)

        # Check if the predicted emotion matches the true emotion
        if predicted_emotion == true_emotion:
            correct += 1

    accuracy = correct / total
    print(f"Accuracy: {accuracy:.2f}")

# Example to predict an emotion and evaluate accuracy
if __name__ == "__main__":
    sample_sentence = "I am feeling really happy today!"
    predict_emotion(sample_sentence)
    
    # Evaluate accuracy using the validation dataset
    evaluate_accuracy('VAL_DATA.csv')


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happiness
Confidence: 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happiness
Confidence: 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 6): Neutral
Confidence: 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 4): Sadness
Confidence: 0.56
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 6): Neutral
Confidence: 0.67
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 6): Neutral
Confidence: 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happiness
Confidence: 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 0): Anger
Confi

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to ParallelMapDatasetV2:802 transformation with iterator: Iterator::Root::Prefetch::ParallelMapV2: indices[0] = 0 is not in [0, 0)
	 [[{{node RaggedGather/RaggedGather}}]] [Op:IteratorGetNext] name: 

In [89]:
# Existing imports remain the same

# Voting classifier
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, sentence):
        votes = []
        
        # Transform the input sentence into TF-IDF features
        features_tfidf = vectorizer.transform([sentence])
        processed_features = preprocess_nltk(sentence)

        for clf in self._classifiers:
            if hasattr(clf, 'predict'):
                prediction = clf.predict(features_tfidf)
                votes.append(str(prediction[0]))
            elif hasattr(clf, 'classify'):
                prediction = clf.classify(processed_features)
                votes.append(str(prediction))
            else:
                # Handle the Keras model
                cleaned_sentence = self.clean_new_text(sentence)
                prediction_input = self.prepare_keras_input(cleaned_sentence)
                prediction = clf.predict(prediction_input)
                
                # Handle prediction for Keras
                predicted_class = np.argmax(prediction, axis=1)[0]
                votes.append(str(predicted_class))

        # Return the most common vote
        return mode(votes)

    def prepare_keras_input(self, text):
        sequences = tokenizer.texts_to_sequences([text])  # Convert text to sequences
        padded_sequences = pad_sequences(sequences, maxlen=max_length)  # Pad sequences
        return padded_sequences  # Ensure this is a 2D array

def evaluate_accuracy(val_data_path):
    val_data = pd.read_csv(val_data_path)
    total = len(val_data)
    correct = 0

    for _, row in val_data.iterrows():
        sentence = row['Sentence']
        true_emotion = row['Emotion']
        predicted_emotion, _ = predict_emotion(sentence)

        # Check if the predicted emotion matches the true emotion
        if predicted_emotion == true_emotion:
            correct += 1

    accuracy = correct / total
    print(f"Accuracy: {accuracy:.2f}")

# Example to predict an emotion and evaluate accuracy
if __name__ == "__main__":
    c=0
    sample_sentence = "I am feeling really happy today!"
    predict_emotion(sample_sentence)
    
    # Evaluate accuracy using the validation dataset
    
    evaluate_accuracy('TEST_DATA.csv')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happiness
Confidence: 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happiness
Confidence: 0.67
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happiness
Confidence: 0.67
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happiness
Confidence: 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happiness
Confidence: 0.78
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 6): Neutral
Confidence: 0.33
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happiness
Confidence: 0.44
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted emotion (index 3): Happi

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to ParallelMapDatasetV2:5122 transformation with iterator: Iterator::Root::Prefetch::ParallelMapV2: indices[0] = 0 is not in [0, 0)
	 [[{{node RaggedGather_1/RaggedGather}}]] [Op:IteratorGetNext] name: 

In [92]:
test_data = pd.read_csv('TEST_DATA.csv')

In [94]:
test.head(10)

,Sentence,Emotion,Length
0,My favourite food is anything I didn't have to...,6,59
1,"Now if he does off himself, everyone will thin...",6,112
2,WHY THE FUCK IS BAYLESS ISOING,0,30
3,To make her feel threatened,2,27
4,Dirty Southern Wankers,0,22
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,5,98
6,Yes I heard abt the f bombs! That has to be wh...,3,117
7,We need more boards and to create a bit more s...,3,82
8,Damn youtube and outrage drama is super lucrat...,3,60
9,It might be linked to the trust factor of your...,6,54
